In [36]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
pd.set_option('display.max_colwidth',None)

data = pd.read_csv('./csv/20200901.csv', low_memory=False)
data['#event_time'] = pd.to_datetime(data['#event_time'])
data.sort_values(by='#event_time', inplace=True)
data = data[~data['#event_name'].isin(['push_send'])]

In [2]:
data.head()

,Unnamed: 0,#user_id,#event_name,#event_time,#app_id,#account_id,#distinct_id,#server_time,#kafka_offset,#province,#device_model,#city,#device_id,#manufacturer,#zone_offset,#app_version,package_name,#title,#screen_name,#os,#os_version,#country_code,#carrier,#ip,#screen_width,mac,#network_type,#lib_version,#duration,#screen_height,#lib,#country,click_page,#resume_from_background,vip,target_uid,myself,tab,rid,room_type,step,room_channel,owner_uid,refer,game_type,is_success,login_type,stage,juben_role_num,juben_id,from,juben_level,is_new,juben_name,juben_title,juben_theme,message_type,total_price,gift_price,gift_id,gift_num,gift_type,gift_name,scene,user_num,ref,is_voice,skill_id,moment_id,comment_id,role_title,role_id,dur,score,is_win,platform,adid,position,person_click,type,room_propery,action,leave_num,if_online,end_time,last_pkg,end_reason,win,vvc,task_name,coin_num,source,task_type,uid,task_id,start_time,cid,item_name,gid,game_role,game_state,game_turn,create_way,room_types,to_uid,room_property,subsidy_money,money,consume_money,consume_type,create_room_uid,role,wait_dur,up_num_people,create_dur,enter_num_people,is_gift_box,is_out_escape,pay_type,dateline,recharge_money,is_online,is_effective,customerservice_pass,operator_pass,operator_efficiency,c_num,c_id,defend_type,defend_id,follow,paystate,channel,campaign,is_send,order_id,reason,cancel_type,page_name,click_type,flow_type,with_type,from_page,title_id,title_name,media_type,tag,to,topic_name,page,push_type_id,content_id,push_type,push_channel,event_name,is_peipei,time,click_tab,num,bp_coupon,bp_item,duration,off_mic_time,on_mic_time,msg_type,total_time,click_button,ms_uid,message_mod,mes_type,profile,share,index,content,pic_num,success_from,play_time,song_id,song_name,rest_time_percent,is_first,gift_scene,newface_uid,newface_vip,onrank,newface_ontime,report_type,show,topic,from_uid,group,game_room_click,chat_room_types,chat_room_type,chat_room_property,combos,is_video,photo_id,is_album,price,topic_page,click,chat_uid,click_position,search_content,follow_uid,$part_event,$part_date,#account_id.1,#user_id.1,birthday,gender,channel.1,register_time,date
61443,551531,747386643689639936,exit_room,2020-09-01 00:00:00.230,42011f0186454b40901619a825564cbb,121414439,53D93923-8915-4F05-92FB-14FB8FBA47F7,2020-09-01 00:00:02.712,218058391,广东,"iPhone11,2",东莞,53D93923-8915-4F05-92FB-14FB8FBA47F7,Apple,8.0,1.0.3,com.wolf.online.ios,NaN,NaN,iOS,13.4.1,CN,NaN,112.97.212.233,375.0,53D93923-8915-4F05-92FB-14FB8FBA47F7,4G,2.4.0,14.215,812.0,iOS,中国,NaN,NaN,4.0,NaN,NaN,NaN,112094879.0,wolf,NaN,auto,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,exit_room,2020-09-01,121414439,747386643689639936,1998-01-01,1.0,apple_wolf,2020-08-24 09:27:33.000,2020-09-01
56701,534359,728377950544207872,coin_income,2020-09-01 00:00:00.479,42011f0186454b40901619a825564cbb,114946779,NaN,2020-09-01 00:00:13.271,217469564,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.3.0,NaN,NaN,tga_php_sdk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,com.imbb.langrenOL.android,NaN,NaN,NaN,收到❌清赠送的小心心x1,12.0,chat-coin,0.0,114946779.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [2]:
data['#event_name'].unique()

array(['exit_room', 'coin_income', 'enter_room', 'ta_app_end',
       'ta_app_start', 'view_chat_page', 'click', 'xs_user_online',
       'game_match', 'room_off_mic', 'chat', 'view_user_homepage',
       'view_flow', 'follow', 'click_flow', 'like', 'game_match_success',
       'room_list_view', 'creat_room', 'send_gift', 'consume', 'dingdong',
       'login', 'coin_consume', 'view_page', 'game_escape', 'game_end',
       'register', 'game_start', 'new_grade', 'ai_guide', 'replay_speech',
       'finish_ai_guide', 'juben_pv_home_card', 'recharge', 'comment',
       'send_moment', 'moment_detail_page', 'juben_click_home_card',
       'moment_cspass', 'juben_pv_home_list', 'view_skill_page',
       'create_room', 'juben_room_create', 'choose_role',
       'quickly_send_gift', 'quit_ai_guide', 'msg_send', 'flow_exposure',
       'view_room', 'on_mic', 'room_public_chat', 'repost',
       'view_flow_time', 'party_page', 'game_grade',
       'juben_click_home_list', 'room_search_click', 'fl

In [30]:
from tqdm import tqdm

event_dict = {}

for i in tqdm(data['#event_name'].unique()):
    x = data.drop(['Unnamed: 0','#user_id','#event_time','#app_id','#account_id','#distinct_id',
                 '#server_time','#kafka_offset','#province','#device_model','#city','#device_id',
                 '#manufacturer','#zone_offset','#app_version','package_name','#os','#os_version',
                 '#country_code','#carrier','#ip','#screen_width','mac','#network_type',
                 '#lib_version','#screen_height','#lib','#country','vip','$part_event','$part_date',
                 '#account_id.1','#user_id.1','birthday','gender','channel.1','register_time','date',
                  ]
                , axis=1)
    x = x[x['#event_name']==i].copy()
    x.dropna(axis=1, how='all', inplace=True)
    value = x.columns.tolist()
    value.remove('#event_name')
    event_dict[i] = value

100%|██████████| 68/68 [00:04<00:00, 16.46it/s]


In [31]:
import pprint
pprint.pprint(event_dict)

{'ai_guide': ['from'],
 'chat': ['target_uid', 'refer', 'message_type', 'ref'],
 'choose_role': ['rid', 'juben_id', 'role_id'],
 'click': ['click_page'],
 'click_flow': ['target_uid',
                'owner_uid',
                'moment_id',
                'click_type',
                'flow_type',
                'page'],
 'coin_consume': ['scene',
                  'last_pkg',
                  'task_name',
                  'coin_num',
                  'source',
                  'task_type',
                  'uid',
                  'task_id',
                  'cid',
                  'item_name',
                  'gid',
                  'is_peipei'],
 'coin_income': ['scene',
                 'last_pkg',
                 'task_name',
                 'coin_num',
                 'source',
                 'task_type',
                 'uid',
                 'task_id',
                 'is_peipei'],
 'comment': ['target_uid', 'moment_id', 'comment_id'],
 'consume': ['rid',
 

In [74]:
data[data['#event_name']=='view_page'][['#event_name','detail']].tail()

#data[data['#event_name']=='room_click']['detail'].unique()

,#event_name,detail
500,view_page,page_name:moment_detail;
479,view_page,page_name:moment_detail;
2121,view_page,page_name:publish;
2109,view_page,page_name:publish;
504,view_page,page_name:moment_detail;


In [38]:
def add_detail(row):
    attrs = event_dict[row['#event_name']]
    string = ''
    for att in attrs:
        string += '%s:%s;'%(att, row[att])
    return string

data['detail'] = data.apply(lambda x: add_detail(x), axis=1)

In [76]:
data[data['#event_name']=='ai_guide'][['#account_id','from']]\
    .drop_duplicates(subset='#account_id',keep='last').rename(columns={'from':'ai_guide_from'}).head()

,#account_id,ai_guide_from
48509,121908290,0.0
54630,118968521,0.0
48510,121928309,0.0
54631,121932880,0.0
48756,114414315,1.0


In [77]:
data[data['#event_name']=='quit_ai_guide'][['#account_id','stage']]\
    .drop_duplicates(subset='#account_id',keep='last').rename(columns={'from':'quit_ai_guide_stage'}).head()

,#account_id,stage
27742,120341081,0.0
19559,121920097,0.0
19560,116371179,0.0
27741,121932880,1.0
19557,121944715,2.0


In [78]:
data[data['#event_name']=='quit_ai_guide'][['#account_id','juben_title']]\
    .drop_duplicates(subset='#account_id',keep='last').rename(columns={'from':'quit_ai_guide_juben_title'}).head()

,#account_id,juben_title
27742,120341081,NaN
19559,121920097,NaN
19560,116371179,NaN
27741,121932880,数字酒店
19557,121944715,数字酒店


In [86]:
events_dict = {'ai_guide':['from']
              ,'quit_ai_guide':['stage','juben_title']
              ,'finish_ai_guide':['juben_title','is_win']
              }

new_features = []
for event, attrs in events_dict.items():
    new_features_e = []
    for attr in attrs:
        group = data[data['#event_name']==event][['#account_id',attr]]\
                .drop_duplicates(subset='#account_id',keep='last')\
                .rename(columns={attr:'%s_LAST_%s'%(event, attr)})\
                .set_index('#account_id')
        new_features_e.append(group)
    new_features.extend(new_features_e)
    
df_features = pd.concat(new_features)

In [87]:
df_features.head()

,ai_guide_LAST_from,quit_ai_guide_LAST_stage,quit_ai_guide_LAST_juben_title,finish_ai_guide_LAST_juben_title,finish_ai_guide_LAST_is_win
#account_id,,,,,
121908290,0.0,NaN,NaN,NaN,NaN
118968521,0.0,NaN,NaN,NaN,NaN
121928309,0.0,NaN,NaN,NaN,NaN
121932880,0.0,NaN,NaN,NaN,NaN
114414315,1.0,NaN,NaN,NaN,NaN


In [89]:
data[data['#event_name']=='click'].groupby(['#account_id','click_page']).size().unstack().head()

click_page,daily_bonus,guess,juben,main_tab_home,main_tab_live,main_tab_msg,main_tab_profile,msg_moment,party,recommend_follow,under,visitor,wolf,wolf12,wolf6
#account_id,,,,,,,,,,,,,,,
107457947,3.0,NaN,1.0,13.0,15.0,12.0,9.0,1.0,NaN,NaN,NaN,NaN,6.0,2.0,1.0
107698826,NaN,NaN,NaN,7.0,5.0,3.0,5.0,1.0,NaN,NaN,NaN,NaN,4.0,14.0,6.0
107705928,NaN,NaN,NaN,2.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,12.0,4.0
110022957,NaN,NaN,NaN,2.0,1.0,2.0,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN
110053403,NaN,NaN,NaN,5.0,NaN,2.0,1.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN
